In [ ]:
import os

db_S_dir = os.environ["DATA"] + "PatImgXAI_data/db3.0.0_S/"
db_L_dir = os.environ["DATA"] + "PatImgXAI_data/db3.0.0_L/"
db_patterns_dir = os.environ["DATA"] + "PatImgXAI_data/db3.0.0/patterns/"


model_dir_root = os.environ["DATA"] + "models/db3.0.0/01_protov5/"
shap_scale_img_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0","shap_scale.png")
yes_pred_img_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0","button_yes.png")
no_pred_img_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0","button_no.png")
yes_small_pred_img_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0","button_yes_small.png")
no_small_pred_img_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0","button_no_small.png")
pos_pred_legend_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0","cf_info_pos.png")
neg_pred_legend_path = os.path.join(os.environ["DATA"] + "PatImgXAI_data/db3.0.0","cf_info_neg.png")
interface_dir = os.environ["DATA"] + "webinterfaces/int05_prototype/"

XAI_DATASET_SIZE = 50

N_JOBS = 20
N_JOBS_GPU = 6

In [ ]:

# Grid division for full image
X_DIVISIONS_L = 15
Y_DIVISIONS_L = 15
X_DIVISIONS_S = 10
Y_DIVISIONS_S = 10

# Grid division of patterns
X_DIVISIONS_PATTERNS = 2
Y_DIVISIONS_PATTERNS = 2

# Probability to generate a geometrical shape at each position in the grid
SHAPE_PROB = 0.5

# Define available shapes
SHAPES = ['circle', 'square', 'triangle']
COLORS  = ["#A33E9A", "#E0B000", "#0C90C0"] # Purple, Yellow, Blue

explict_colors_dict = {
    "#A33E9A": "purple",
    "#E0B000": "yellow",
    "#0C90C0": "blue"
}

In [ ]:
from xaipatimg.datagen.dbimg import load_db

db_patterns = load_db(db_patterns_dir)

In [ ]:
pattern_2sym_keys = []
pattern_3sym_keys = []
for k, v in db_patterns.items():
    if len(v["content"]) == 2:
        pattern_2sym_keys.append(k)
    if len(v["content"]) == 3:
        pattern_3sym_keys.append(k)


In [ ]:
datasets_path_L = os.path.join(db_L_dir, "datasets", "01_protov5")
datasets_path_S = os.path.join(db_S_dir, "datasets", "01_protov5")

In [ ]:
from xaipatimg.datagen.gendataset import generic_rule_pattern_exactly_1_time_exclude_more, \
    generic_rule_pattern_exactly_N_times

rules_data_L = [
    {"name": "hard1_2sym", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_L,
                                                                                                     "y_division_full": Y_DIVISIONS_L,
                                                                                                     "x_division_pattern": X_DIVISIONS_PATTERNS,
                                                                                                     "y_division_pattern": Y_DIVISIONS_PATTERNS,
                                                                                                     "consider_rotations": False},
     "question": "Is the pattern in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 10, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "filter_on_dim": [X_DIVISIONS_L, Y_DIVISIONS_L], "pattern_id": pattern_2sym_keys[3]},

    {"name": "hard2_3sym", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_L,
                                                                                                     "y_division_full": Y_DIVISIONS_L,
                                                                                                     "x_division_pattern": X_DIVISIONS_PATTERNS,
                                                                                                     "y_division_pattern": Y_DIVISIONS_PATTERNS,
                                                                                                     "consider_rotations": False},
     "question": "Is the pattern in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 10, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "filter_on_dim": [X_DIVISIONS_L, Y_DIVISIONS_L], "pattern_id": pattern_3sym_keys[1]},

    {"name": "hard3_2sym_rot", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_L,
                                                                                                     "y_division_full": Y_DIVISIONS_L,
                                                                                                     "x_division_pattern": X_DIVISIONS_PATTERNS,
                                                                                                     "y_division_pattern": Y_DIVISIONS_PATTERNS,
                                                                                                     "consider_rotations": True},
     "question": "Is the pattern or its left/right rotations in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 10, "pos_llm_scaffold": "", "w": "", "filter_on_dim": [X_DIVISIONS_L, Y_DIVISIONS_L], "pattern_id": pattern_2sym_keys[4]},

    {"name": "hard4_2sym_3times", "gen_fun": generic_rule_pattern_exactly_N_times, "gen_kwargs": {"x_division_full": X_DIVISIONS_L,
                                                                                                     "y_division_full": Y_DIVISIONS_L,
                                                                                                     "x_division_pattern": X_DIVISIONS_PATTERNS,
                                                                                                     "y_division_pattern": Y_DIVISIONS_PATTERNS,
                                                                                                     "N": 3},
     "question": "Is the pattern exactly 3 times in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 10, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "filter_on_dim": [X_DIVISIONS_L, Y_DIVISIONS_L], "pattern_id": pattern_2sym_keys[5]},

]

In [ ]:
from xaipatimg.datagen.gendataset import generic_rule_pattern_exactly_1_time_exclude_more, \
    generic_rule_pattern_exactly_N_times

rules_data_S = [
    {"name": "easy1_2sym", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_S,
                                                                                                     "y_division_full": Y_DIVISIONS_S,
                                                                                                     "x_division_pattern": X_DIVISIONS_PATTERNS,
                                                                                                     "y_division_pattern": Y_DIVISIONS_PATTERNS,
                                                                                                     "consider_rotations": False},
     "question": "Is the pattern in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 10, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "filter_on_dim": [X_DIVISIONS_S, Y_DIVISIONS_S], "pattern_id": pattern_2sym_keys[0]},

    {"name": "easy2_3sym", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_S,
                                                                                                     "y_division_full": Y_DIVISIONS_S,
                                                                                                     "x_division_pattern": X_DIVISIONS_PATTERNS,
                                                                                                     "y_division_pattern": Y_DIVISIONS_PATTERNS,
                                                                                                     "consider_rotations": False},
     "question": "Is the pattern in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 10, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "filter_on_dim": [X_DIVISIONS_S, Y_DIVISIONS_S], "pattern_id": pattern_3sym_keys[0]},

    {"name": "easy3_2sym_rot", "gen_fun": generic_rule_pattern_exactly_1_time_exclude_more, "gen_kwargs": {"x_division_full": X_DIVISIONS_S,
                                                                                                     "y_division_full": Y_DIVISIONS_S,
                                                                                                     "x_division_pattern": X_DIVISIONS_PATTERNS,
                                                                                                     "y_division_pattern": Y_DIVISIONS_PATTERNS,
                                                                                                         "consider_rotations": True},
     "question": "Is the pattern or its left/right rotations in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 10, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "filter_on_dim": [X_DIVISIONS_S, Y_DIVISIONS_S], "pattern_id": pattern_2sym_keys[1]},

    {"name": "easy4_2sym_2times", "gen_fun": generic_rule_pattern_exactly_N_times, "gen_kwargs": {"x_division_full": X_DIVISIONS_S,
                                                                                                     "y_division_full": Y_DIVISIONS_S,
                                                                                                     "x_division_pattern": X_DIVISIONS_PATTERNS,
                                                                                                     "y_division_pattern": Y_DIVISIONS_PATTERNS,
                                                                                                     "N": 2},
     "question": "Is the pattern exactly 2 times in the image?", "target_acc" : 0.85, "shown_acc" : 0.85, "samples_interface": 10, "pos_llm_scaffold": "", "neg_llm_scaffold": "", "filter_on_dim": [X_DIVISIONS_S, Y_DIVISIONS_S], "pattern_id": pattern_2sym_keys[2]},

]

In [ ]:
from xaipatimg.ml.xai import generate_shap_resnet18, generate_counterfactuals_resnet18_random_approach, \
    create_xai_index
from tqdm import tqdm

def generate_explanations(rules_data, db_dir, datasets_dir_path):

    for rule_idx in tqdm(range(len(rules_data))):

        model_dir = os.path.join(model_dir_root, rules_data[rule_idx]["name"])
        dataset_filename = rules_data[rule_idx]["name"] + "_test.csv"
        generic_rule_fun = rules_data[rule_idx]["gen_fun"]
        generic_rule_fun_kwargs = rules_data[rule_idx]["gen_kwargs"]
        xai_output_paths = {
            "shap" : "shap",
            "cf" : "cf",
        }

        generic_rule_fun_kwargs["pattern_content"] = db_patterns[rules_data[rule_idx]["pattern_id"]]["content"]

        generate_shap_resnet18(db_dir, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename,
                               model_dir=model_dir, xai_output_path=os.path.join(model_dir, xai_output_paths["shap"]),
                               yes_pred_img_path=yes_pred_img_path, no_pred_img_path=no_pred_img_path, device="cuda:0", n_jobs=N_JOBS,
                               dataset_size=XAI_DATASET_SIZE, masker="ndarray", shap_scale_img_path=shap_scale_img_path)

        # generate_counterfactuals_resnet18_random_approach(db_dir, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename,
        #                                                   model_dir=model_dir,
        #                                                   xai_output_path=os.path.join(model_dir, xai_output_paths["cf"]),
        #                                                   yes_pred_img_path=yes_pred_img_path, no_pred_img_path=no_pred_img_path,
        #                                                   shapes=SHAPES, colors=COLORS, empty_probability=1-SHAPE_PROB,
        #                                                   max_depth=10, nb_tries_per_depth=2000, generic_rule_fun=generic_rule_fun,
        #                                                   devices=["cuda:0", "cuda:1"], n_jobs=N_JOBS_GPU,
        #                                                   dataset_size=XAI_DATASET_SIZE,pos_pred_legend_path=pos_pred_legend_path,
        #                                                   neg_pred_legend_path=neg_pred_legend_path,
        #                                                   **generic_rule_fun_kwargs)

        create_xai_index(db_dir, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename, model_dir=model_dir,
                         xai_dirs=xai_output_paths, dataset_size=XAI_DATASET_SIZE, device="cuda:0")


In [ ]:
generate_explanations(rules_data_S, db_S_dir, datasets_path_S)

In [ ]:
generate_explanations(rules_data_L, db_L_dir, datasets_path_L)

In [ ]:
# from transformers import AutoTokenizer
# from transformers import AutoModelForCausalLM
# import csv
# from xaipatimg.ml.xai import generate_LLM_explanations, create_xai_index
# from tqdm import tqdm
#
# model_id = "openai/gpt-oss-20b"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# llm_model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     torch_dtype="auto",
# )
#
# for rule_idx in tqdm(range(len(rules_data))):
#
#     model_dir = os.path.join(model_dir_root, rules_data[rule_idx]["name"])
#     dataset_filename = rules_data[rule_idx]["name"] + "_test.csv"
#
#     # Extracting the subset of indices of samples selected for the experimental interface, in order to ease the cost of calculation
#     interface_content_path = os.path.join(interface_dir, "res", "tasks", f"{rules_data[rule_idx]["name"]}_content.csv")
#     interface_selected_idx = [int(row["og_idx"]) for row in list(csv.DictReader(open(interface_content_path), delimiter=','))]
#
#     xai_output_paths = {
#         "shap" : "shap",
#         "cf" : "cf",
#         "llm" : "llm",
#     }
#     generate_LLM_explanations(db_dir, db, datasets_dir_path=datasets_dir_path, dataset_filename=dataset_filename,
#                               model_dir=model_dir, llm_model=llm_model, llm_tokenizer=tokenizer,
#                               xai_output_path=os.path.join(model_dir, xai_output_paths["llm"]),
#                               explicit_colors_dict=explict_colors_dict, question=rules_data[rule_idx]["question"],
#                               yes_pred_img_path=yes_pred_img_path, no_pred_img_path=no_pred_img_path,
#                               yes_pred_img_path_small=yes_small_pred_img_path, no_pred_img_path_small=no_small_pred_img_path,
#                               device="cuda:0", dataset_size=XAI_DATASET_SIZE, only_for_index=interface_selected_idx,
#                               path_to_counterfactuals_dir_for_model_errors=os.path.join(model_dir, xai_output_paths["cf"]),
#                               pos_llm_scaffold=rules_data[rule_idx]["pos_llm_scaffold"], neg_llm_scaffold=rules_data[rule_idx]["neg_llm_scaffold"])
#
#     create_xai_index(db_dir, dataset_filename=dataset_filename, model_dir=model_dir, xai_dirs=xai_output_paths, dataset_size=XAI_DATASET_SIZE, device="cuda:0")
